In [16]:
import urllib
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://www.vivino.com/explore?e=eJwrL4mOtTVUKwdRRgAgBwRK&page='
url_vinok = 'https://www.vivino.com/ES/es'
data = []

# Number of pages to scrape
num_pages = 100

for page in range(1, num_pages + 1):
    url = base_url + str(page)
    print(f"Accediendo a la página {page}: {url}")
    try:
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html, 'html.parser')
        tags = soup.find_all('a', class_='anchor_anchor__m8Qi- wineCard__cardLink--3F_uB wineCard__notSuperCard--268Xo')

        for tag in tags:
            new_url = url_vinok + tag.get('href')
            try:
                if new_url.startswith('http'):
                    html2 = urllib.request.urlopen(new_url)
                    soup2 = BeautifulSoup(html2)
                    winery = soup2.find('span', class_='headline')
                    name = soup2.find('span', class_='vintage')
                    price = soup2.find('span', class_='purchaseAvailability__currentPrice--3mO4u')
                    score = soup2.find('div', class_='vivinoRating_averageValue__uDdPM')
                    country = soup2.find('a', {'data-cy': 'breadcrumb-country'})
                    wine_type = soup2.find('a', {'data-cy': 'breadcrumb-winetype'})
                    pairing = soup2.findAll('a', class_='foodPairing__imageContainer--2CtYR')
                    flavours = soup2.find('div', class_='tasteNote__popularKeywords--1gIa2')
                    img_src = soup2.find('picture', class_='bottleShot').find('img', class_='image')['src']
                    price_quality = soup2.find('span', class_='MuiTypography-root MuiTypography-body1 vivino-mui-jl3vsy-MuiTypography-root-labelLarge')

                    # Encuentra la tabla
                    table = soup2.find('table', {'class': 'wineFacts__wineFacts--2Ih8B'})

                    # Extrae las filas de la tabla
                    table_data = {}
                    if table:
                        rows = table.find_all('tr')
                        for row in rows:
                            cells = row.find_all(['td', 'th'])
                            if len(cells) == 2:
                                key = cells[0].get_text(strip=True)
                                value = cells[1].get_text(strip=True)
                                table_data[key] = value

                    wine_name, year = name.text.strip().split('\n\n')
                    food_list = [food.text for food in pairing]
                    if flavours:
                        flavours_list = [flavour.text for flavour in flavours]
                    else:
                        flavours_list = []

                    # Añade los datos de la tabla al diccionario
                    wine_data = {
                        'winery': winery.text.strip() if winery else None,
                        'wine_name': wine_name,
                        'year': year,
                        'price': price.text.strip() if price else None,
                        'score': score.text.strip() if score else None,
                        'country': country.text if country else None,
                        'wine_type': wine_type.text if wine_type else None,
                        'pairing': ', '.join(food_list) if food_list else None,
                        'flavours': ', '.join(flavours_list) if flavours_list else None,
                        'picture': img_src,
                        'price_quality': price_quality.text.strip() if price_quality else None
                    }
                    wine_data.update(table_data)

                    data.append(wine_data)

            except Exception as e:
                print(f"Error al acceder a {new_url}: {e}")

    except Exception as e:
        print(f"Error al acceder a la página {page}: {e}")

df = pd.DataFrame(data, columns=['winery', 'wine_name', 'year', 'price', 'score', 'country', 'wine_type', 'pairing', 'flavours', 'picture', 'price_quality', 'Uvas', 'Región', 'Estilo de vino', 'Contenido de alcohol'])

Accediendo a la página 1: https://www.vivino.com/explore?e=eJwrL4mOtTVUKwdRRgAgBwRK&page=1
Error al acceder a la página 1: HTTP Error 429: Too Many Requests
Accediendo a la página 2: https://www.vivino.com/explore?e=eJwrL4mOtTVUKwdRRgAgBwRK&page=2
Error al acceder a la página 2: HTTP Error 429: Too Many Requests
Accediendo a la página 3: https://www.vivino.com/explore?e=eJwrL4mOtTVUKwdRRgAgBwRK&page=3
Error al acceder a la página 3: HTTP Error 429: Too Many Requests
Accediendo a la página 4: https://www.vivino.com/explore?e=eJwrL4mOtTVUKwdRRgAgBwRK&page=4
Error al acceder a la página 4: HTTP Error 429: Too Many Requests
Accediendo a la página 5: https://www.vivino.com/explore?e=eJwrL4mOtTVUKwdRRgAgBwRK&page=5
Error al acceder a la página 5: HTTP Error 429: Too Many Requests
Accediendo a la página 6: https://www.vivino.com/explore?e=eJwrL4mOtTVUKwdRRgAgBwRK&page=6
Error al acceder a la página 6: HTTP Error 429: Too Many Requests
Accediendo a la página 7: https://www.vivino.com/explore?e

KeyboardInterrupt: 

In [18]:
df.head()

,winery,wine_name,year,price,score,country,wine_type,pairing,flavours,picture,price_quality,Uvas,Región,Estilo de vino,Contenido de alcohol


In [19]:
df.dtypes

winery                  object
wine_name               object
year                    object
price                   object
score                   object
country                 object
wine_type               object
pairing                 object
flavours                object
picture                 object
price_quality           object
Uvas                    object
Región                  object
Estilo de vino          object
Contenido de alcohol    object
dtype: object

In [20]:
df.to_csv('vivino.csv', index=False)